In [ ]:
%matplotlib inline

In [ ]:
import os
from itertools import product
import numpy as np
import pandas as pd

from keras.optimizers import RMSprop
from kerasdcgan.etl import read_etl6_data, data2array
from kerasdcgan.models import DCGAN
from kerasdcgan.plot import plot_images

### Read training data

In [ ]:
etl6_filepath = '/root/share/personal/data/ETL/ETL6/'
npz_filepath = '/tmp/etl6.npz'

if os.path.exists(npz_filepath):
    dataz = np.load(npz_filepath)
    x_all, y_all = dataz['x_all'], dataz['y_all']
else:
    data = read_etl6_data(etl6_filepath)
    x_all, y_all = data2array(data, new_shape=(28, 28))
    np.savez(file=npz_filepath, x_all=x_all, y_all=y_all)

# Get katakana images
katakana = [''.join(e) for e in product(' KSTNHMYRW', 'AIUEO')] + [' N']  # Includes obsolete characters
katakana_idx = [i for i, label in enumerate(y_all) if label in katakana]
x_train = x_all[katakana_idx]

### Build DCGAN model

In [ ]:
a_optimizer = RMSprop(lr=0.0002, decay=6e-8)
ga_optimizer = RMSprop(lr=0.0001, decay=3e-8)

dcgan = DCGAN(g_input_dim=100, g_filters=512, g_kernel_size=5,
              g_dropout_rate=0.4, g_distribution='gaussian',
              a_filters=512, a_kernel_size=5, a_leakyrelu_alpha=0.2, a_dropout_rate=0.4,
              a_optimizer=a_optimizer,
              ga_optimizer=ga_optimizer)
dcgan.build()
dcgan.summary()

### Train DCGAN

#### Fixed noise vectors to check generator progress

In [ ]:
noise_samples = dcgan.generate_noise(25)

#### Main training loop

In [ ]:
# Image batch generator
def image_batch_generator(x, batch_size):
    n = x.shape[0]
    while 1:
        idx = np.random.shuffle(np.array(range(n)))
        for batch in range(n // batch_size):
            i1, i2 = batch*batch_size, (batch + 1)*batch_size
            yield x[i1:i2]


# For printing progress
def series_to_string(series):
    string = series.to_string().split('\n')
    string = [': '.join(s.split()) for s in string]
    string = '    '.join(string)
    return string

In [ ]:
steps = 5000
batch_size = 256
generator = image_batch_generator(x_train, batch_size)

df = pd.DataFrame(columns=['ad_loss', 'ad_acc', 'stacked_loss', 'stacked_acc'])

In [ ]:
for step in range(steps):
    x_train_batch = next(generator)
    ad_metrics, stacked_metrics = dcgan.train_on_batch(x_train_batch, freeze_discriminator=True)
    
    df.loc[step, ['ad_loss', 'ad_acc', 'stacked_loss', 'stacked_acc']] = ad_metrics + stacked_metrics
    
    if step % 500 == 0:
        print(step, series_to_string(df.loc[step]))
        filename = 'fake_images_%s.png' % str(step).zfill(6)
        generated_images = dcgan.generate(noise_samples)
        plot_images(generated_images, filename, cols=5)

### Generate images

In [ ]:
generated_images = dcgan.generate(25)
plot_images(generated_images, cols=5)

### Plot loss/accuracy progress

In [ ]:
df.plot(figsize=(20, 10))
plt.ylim([0, 5])
plt.show()